# Dataset Partitioning

In [1]:
import os
import re

from collections import defaultdict

import random

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import wikipedia

from nltk import word_tokenize, sent_tokenize, pos_tag
from nltk.corpus import stopwords
from razdel import sentenize

import pymorphy2 as pm2

In [2]:
SEED = 42

random.seed(SEED)

### 0. Introduction

На предыдущем шаге мы создали датасет для классификации предложений корпуса на подходящие/неподходящие в генерируемые абстракты.

In [3]:
corpus = pd.read_csv('../data/data_frames/sentence_suitability_for_article.csv', index_col=0)

corpus

,text,related bigram,suitability
0,Спектры пропускания образцов в диапазоне длин ...,длина_волна,0
1,Для нитрата никеля(II) зависимости изменения о...,длина_волна,0
2,Зависимость инкремента от длины волны и угла б...,длина_волна,0
3,График зависимости инкремента от длины волны и...,длина_волна,0
4,Изменение отражения для каждой длины волны опр...,длина_волна,0
...,...,...,...
8059,По мере удаления от апертуры фронт волны прини...,плоский_волна,1
8060,Уравнение любой волны является решением диффер...,плоский_волна,1
8061,Правильность этой формулы следует из формулы Э...,плоский_волна,1
8062,Но так как в нашем реальном мире не существует...,плоский_волна,1


Теперь разделим его на train/test части. Будем придерживаться пропорции 2 к 1. 

### 1. Organizing the Split

Кол-во терминов в датасете:

In [4]:
max_term_id = corpus['related bigram'].unique().size
max_term_id

50

Сопоставим каждому термину его ID:

In [5]:
unique_terms = corpus['related bigram'].unique()
term_ids = {i: unique_terms[i] for i in range(max_term_id)}

term_ids

{0: 'длина_волна',
 1: 'решение_уравнение',
 2: 'система_уравнение',
 3: 'электрический_поле',
 4: 'комнатный_температура',
 5: 'твёрдый_тело',
 6: 'постановка_задача',
 7: 'средний_значение',
 8: 'граница_раздел',
 9: 'фазовый_переход',
 10: 'уравнение_движение',
 11: 'показатель_преломление',
 12: 'функция_распределение',
 13: 'диэлектрический_проницаемость',
 14: 'большой_число',
 15: 'институт_физика',
 16: 'порядок_величина',
 17: 'дифференциальный_уравнение',
 18: 'плотность_ток',
 19: 'волновой_число',
 20: 'волновой_вектор',
 21: 'кинетический_энергия',
 22: 'скорость_свет',
 23: 'кристаллический_структура',
 24: 'математический_модель',
 25: 'погрешность_измерение',
 26: 'коэффициент_отражение',
 27: 'коэффициент_диффузия',
 28: 'научный_школа',
 29: 'волновой_функция',
 30: 'численный_метод',
 31: 'коэффициент_поглощение',
 32: 'объект_исследование',
 33: 'окружающий_среда',
 34: 'скорость_звук',
 35: 'энергия_связь',
 36: 'спектр_поглощение',
 37: 'время_релаксация',
 38: 'т

**Важно!** Предложения для 1 термина могут быть или только в train, или только в test.

Общая стратегия разделения:
1. берем случайный термин;
2. пытаемся добавить предложения с ним в test часть - если кол-во предложений в test стало больше допустимого предела, пробуем добавить другой термин. Иначе - добавляем. Если и после этого предложений меньше минимального порога, повторяем шаги 1-2;
3. если кол-во предложений в test попало в искомый диапазон, добавляем все оставшиеся предложения в train часть.

In [6]:
TEST_SIZE = 0.33
TEST_ERROR_MARGIN = 0.03

In [7]:
test_item_max_limit = round((TEST_SIZE + TEST_ERROR_MARGIN) * len(corpus))
test_item_max_limit

2903

In [8]:
test_item_min_limit = round((TEST_SIZE - TEST_ERROR_MARGIN) * len(corpus))
test_item_min_limit

2419

In [9]:
test_terms, test_count = set(), 0

In [10]:
viewed_term_ids = set()

while test_count < test_item_min_limit:
    # finish if no suitable candidate 
    if len(viewed_term_ids) == max_term_id:
        break
    
    # choose next candidate
    current_term_id = random.randint(0, max_term_id - 1)
    if current_term_id in viewed_term_ids:
        continue
    viewed_term_ids.add(current_term_id)
    current_term = term_ids[current_term_id]

    # inspect partition
    partition = corpus[corpus['related bigram'] == current_term]
    if test_count + len(partition) > test_item_max_limit:
        continue
    test_count += len(partition)
    test_terms.add(current_term)

In [11]:
train_data, test_data = [], []

for row in corpus.itertuples():
    if row[2] in test_terms:
        test_data.append(row)
    else:
        train_data.append(row)

In [12]:
test_df = pd.DataFrame(data=test_data, columns=['1', 'text', 'related_term', 'suitability']).drop(columns='1')
test_df

,text,related_term,suitability
0,"Решение уравнения поля в среде, ограниченной ц...",решение_уравнение,0
1,Однако аналитическое и численное решение уравн...,решение_уравнение,0
2,Однако аналитическое и численное решение уравн...,решение_уравнение,0
3,ВОЛНОВЫЕ РЕШЕНИЯ УРАВНЕНИЯ ЭРОЗИИ Для исследов...,решение_уравнение,0
4,Даже в таком простом случае решение уравнения ...,решение_уравнение,0
...,...,...,...
2416,"Тем не менее, существует теорема о том, что ря...",теория_возмущение,1
2417,"Это означает, что, начиная с некоторого (на пр...",теория_возмущение,1
2418,"Несмотря на свою кажущуюся универсальность, ме...",теория_возмущение,1
2419,Примерами могут являться инстантонные эффекты ...,теория_возмущение,1


In [13]:
train_df = pd.DataFrame(data=train_data, columns=['1', 'text', 'related_term', 'suitability']).drop(columns='1')
train_df

,text,related_term,suitability
0,Спектры пропускания образцов в диапазоне длин ...,длина_волна,0
1,Для нитрата никеля(II) зависимости изменения о...,длина_волна,0
2,Зависимость инкремента от длины волны и угла б...,длина_волна,0
3,График зависимости инкремента от длины волны и...,длина_волна,0
4,Изменение отражения для каждой длины волны опр...,длина_волна,0
...,...,...,...
5638,По мере удаления от апертуры фронт волны прини...,плоский_волна,1
5639,Уравнение любой волны является решением диффер...,плоский_волна,1
5640,Правильность этой формулы следует из формулы Э...,плоский_волна,1
5641,Но так как в нашем реальном мире не существует...,плоский_волна,1


Сохраним полученные подвыборки:

In [14]:
train_df.to_csv('../data/data_frames/datasets/suitability_classification/train.csv')
test_df.to_csv('../data/data_frames/datasets/suitability_classification/test.csv')

### 2. Conclusion

#### 2.1 Notes

- посмотрел, **отличаются ли выжимки** по терминам в зависимости от их **частотных характеристик** (по сути, единственная имеющаяся у нас информация). Вердикт - **нет**, тут все чистой воды рандом: у частотной биграммы может быть крохотная статья на википедии, а у редкой - большая и информативная;
- с учетом предыдущего соображения решил **выбирать термины** в тестовую часть **рандомно** и просто регулировать, чтобы примерно **сохранялась желаемая пропорция 2 к 1 между train и test** и **все предложения каждого термина попадали только в одну из 2 частей**;

#### 2.2 Improvements

- **TO-DO:**
    1. ;
    2. ;
    3. .